In [9]:
### 📘 Cell 1: Import necessary packages
include("include.jl");

In [2]:
### 📘 Cell 2: Load the JLD2 file
# Replace "your_file.jld2" with your actual file path
data = MyOriginalPortfolioDataSet()


Dict{String, Any} with 1 entry:
  "dataset" => Dict{String, DataFrame}("TPR"=>1828×8 DataFrame…

In [3]:
### 📘 Cell 3: Extract the DataFrame from the loaded file
# Replace "your_variable_name" with the actual variable name in the JLD2 file
original_dataset = data["dataset"]


Dict{String, DataFrame} with 515 entries:
  "TPR"  => 1828×8 DataFrame…
  "EMR"  => 2792×8 DataFrame…
  "CTAS" => 2792×8 DataFrame…
  "HSIC" => 2792×8 DataFrame…
  "KIM"  => 2792×8 DataFrame…
  "PLD"  => 2792×8 DataFrame…
  "IEX"  => 2792×8 DataFrame…
  "KSU"  => 2001×8 DataFrame…
  "BAC"  => 2792×8 DataFrame…
  "CBOE" => 2792×8 DataFrame…
  "EXR"  => 2792×8 DataFrame…
  "NCLH" => 2792×8 DataFrame…
  "CVS"  => 2792×8 DataFrame…
  "DRI"  => 2792×8 DataFrame…
  "DTE"  => 2792×8 DataFrame…
  "ZION" => 2792×8 DataFrame…
  "AVY"  => 2792×8 DataFrame…
  "EW"   => 2792×8 DataFrame…
  "EA"   => 2792×8 DataFrame…
  "NWSA" => 2792×8 DataFrame…
  "BBWI" => 884×8 DataFrame…
  "CAG"  => 2792×8 DataFrame…
  "GPC"  => 2792×8 DataFrame…
  "FCX"  => 2792×8 DataFrame…
  "GILD" => 2792×8 DataFrame…
  ⋮      => ⋮

In [4]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow

2792

In [5]:
original_dataset["AAPL"]

Row,volume,volume_weighted_average_price,open,close,high,low,timestamp,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Int64
1,3.93215e8,19.4749,19.745,19.3207,19.775,19.3011,2014-01-03T05:00:00,148584
2,4.13437e8,19.3213,19.1946,19.4261,19.5286,19.0571,2014-01-06T05:00:00,131664
3,3.17731e8,19.3329,19.44,19.2871,19.4986,19.2116,2014-01-07T05:00:00,107327
4,2.58747e8,19.4038,19.2432,19.4093,19.4843,19.2389,2014-01-08T05:00:00,86874
5,2.79621e8,19.2943,19.5286,19.1614,19.5307,19.1196,2014-01-09T05:00:00,93562
6,3.05283e8,19.0659,19.2796,19.0336,19.3143,18.9682,2014-01-10T05:00:00,113063
7,3.79443e8,19.1801,18.9254,19.1332,19.375,18.9243,2014-01-13T05:00:00,130227
8,3.34937e8,19.4033,19.2221,19.5139,19.5261,19.2021,2014-01-14T05:00:00,114856
9,3.9389e8,19.9105,19.7686,19.9057,20.0071,19.7021,2014-01-15T05:00:00,136942


In [6]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end

In [7]:
list_of_all_tickers = keys(dataset) |> collect |> x->sort(x)

424-element Vector{String}:
 "A"
 "AAL"
 "AAP"
 "AAPL"
 "ABBV"
 "ABT"
 "ACN"
 "ADBE"
 "ADI"
 "ADM"
 "ADP"
 "ADSK"
 "AEE"
 ⋮
 "WST"
 "WU"
 "WY"
 "WYNN"
 "XEL"
 "XOM"
 "XRAY"
 "XYL"
 "YUM"
 "ZBRA"
 "ZION"
 "ZTS"

In [8]:
### 📘 Cell 4: Split and save each ticker's data into two dictionaries
train_start = DateTime(2014, 1, 1)
train_end = DateTime(2023, 12, 31, 23, 59, 59)

train_dataset = Dict{String, DataFrame}()
test_dataset = Dict{String, DataFrame}()

for ticker in list_of_all_tickers
    df = dataset[ticker]
    df.timestamp = DateTime.(df.timestamp)  # ensure the timestamp column is DateTime

    train_data = filter(row -> train_start <= row.timestamp <= train_end, df)
    test_data = filter(row -> row.timestamp > train_end, df)

    train_dataset[ticker] = train_data
    test_dataset[ticker] = test_data
end

train_file = joinpath(_PATH_TO_DATA, "train_dataset_2014_2023.jld2")
test_file = joinpath(_PATH_TO_DATA, "test_dataset_2024_onward.jld2")

@save train_file train_dataset
@save test_file test_dataset